## 목표: 와인 성분 데이터를 이용해 품질예측모형 개발

## 학습목표
- 처음부터 마지막 단계까지 데이터 분석 실습을 수행하며 일반적인 데이터 분석 과정을 이해한다.
- 데이터 분석 각 단계의 목적 및 필요성을 이해한다.
- 와인 품질에 영향을 미치는 요인을 회귀분석을 통해 도출한다.
- 머신러닝을 활용한 와인 품질 예측 모형을 개발해 본다.

#### 작업환경 설정

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings('ignore')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 8

pd.set_option('max_rows', 30)
pd.set_option('max_columns', 15)

## 1단계: 데이터 준비

### 여러 데이터를 한꺼번에 불러와서 합치고 정리하기

- 여러 개의 데이터 프레임을 하나의 데이터 프레임으로 합치기: concat()

- 그런데 white와 red가 데이터상에 구분이 안됨

> **Q. type이라는 coloumn을 추가해서 red=0이라고 기록하고, white=1이라고 기록해서 합치기**

In [19]:
wine.quality.unique()

array([5, 6, 7, 4, 8, 3, 9], dtype=int64)

In [20]:
sorted(wine.quality.unique())

[3, 4, 5, 6, 7, 8, 9]

> **Q. 각각의 퀄리티별 와인 수가 몇개일까?

In [ ]:
# quality가 9등급인 와인의 데이터를 보고 싶다면?

## 탐색적 분석(EDA)
- 기술적 통계 분석
- 시각화

In [23]:
wine.groupby('type').quality.describe()

,count,mean,std,min,25%,50%,75%,max
type,,,,,,,,
0,1599.0,5.636023,0.807569,3.0,5.0,6.0,6.0,8.0
1,4898.0,5.877909,0.885639,3.0,5.0,6.0,6.0,9.0


> **와인 종류에 따라 품질의 분포가 다를까? displot()**
- white의 quality만
- red의 quality만 보기

In [ ]:
sns.distplot(red,label='Red Wine')
sns.distplot(white, label='White Wine')
plt.legend()
plt.title("Distribution of Quality")
plt.xlabel("Quality Score")
plt.ylabel('Density')

- Lesson Learned
- 두 그룹의 와인 모두 품질은 근사적인 정규분포를 따름
- distplot을 그려보면 단순한 기술통계(describe())보다 두 그룹의 분포에 대한 세부 정보를 더 알 수 있음

### T검정
- 와인 종류에 따른 품질의 차이가 통계적으로 유의한지 알기 위해
- 그룹별 품질의 평균과 표준편차를 이용: groupby(), describe(), agg()
- 두 그룹 모두 std 표준편차가 적으면, 합동분산 t검정 실시
- T값, p-value: <0.05 (95%이상 신뢰수준으로 두 그룹의 품질 차이가 통계적으로 유의함)

In [3]:
# print('T값은 %.3f이고 pvalue는 %.7f' %(tstat, pvalue))

### 상관관계분석

- 밀도, 결합산, 휘발산, 잔여설탕, 총이산화황 품질과 음의 상관관계, 특히 밀도
- 알코올, 이산화황, 산성도, 구연산은 품질과 양의 상관관계, 특히 알코올

### pairplot(): 변수들의 관계(산점도)
- 모든 샘플에 대해 점을 찍음 6400개? memory error
- 그래서 sampling, red 200개, white 200개 정도씩만 점 찍음

In [34]:
def sampling(data, n=200, replace=False):
    return data.loc[np.random.choice(data.index, size=n, replace = replace)]

> **원본 데이터인 wine에서 wine_sample 샘플링이 된 샘플인지 아닌지 표시** 
>- np.where(조건문, True일 때 값, False일 때 값), excel의 if와 유사

### 통계분석- 선형회귀 분석
- 각 독립변수가 종속변수에 미치는 영향을 측정
- 설명 & 예측의 목적

- y = ax1+bx2+cx3 + d
- quality = a* alcohol + b* sugar + c* density +...+d

- 독립변수들의 측정단위가 저마다 다 달라서 이 같은 문제가 발생(density는 거의 변수들간 차이가 없음)
- 그래서 scaling이 필요
- 이를 표준화라고 함(정규분포를 표준정규분포화하는 것과 동일)
- 측정값에서 평균에서 빼주고 다시 표준편차로 나눔(Z분포화)

#### 독립변수의 표준화

In [59]:
model_standard = ols(wine_formula, data=wine_standard).fit()

In [4]:
model_standard.summary()

### AI 모델 개발
- Regression(회귀분석); 데이터를 이용해 설명과 예측이 가능하지만 설명이 주요 목적
- AI Machine Learning; 대량의 데이터를 이용한 예측이 주요 목적

#### 머신러닝의 학습과정
- 트레인(훈련)데이터와 테스트데이터, 두 종류가 필요
>- 트레인(Train)데이터: 머신이 학습(공부)하기 위한 데이터(문제와 답)
>- 테스트(Test)데이터: 머신이 트레인데이터로 학습한 후 학습을 얼마나 잘 했는지 테스트(시험)하기 위한 데이터(문제)

- train 데이터의 X값들: X_train(머신이 학습할 문제)
- train 데이터의 y값들: y_train(머신이 학습할 문제의 답)
- model.fit(X_train, y_train) → 트레인데이터로 학습(공부)하라는 명령어
- test 데이터 X값들: X_test(머신이 답을 맞춰야 할 문제)
- test 데이터 y값들: y_test(머신이 X_test데이터로 예측한 값을 비교해 볼 정답)

In [ ]:
# 전체 데이터에서 X, y를 나눔

- Model 학습

In [79]:
model.fit(X_train, y_train)

LinearRegression()

- 예측

- 평가

#### 다음 단계의 내용은?
- R^2 값을 높일 수 있는 독립변수들을 발굴해서 분석에 추가